# 3D Information Processing
Obtain a 3D shape from images taken by multiple cameras

In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import pandas as pd
import numpy as np
from camera import Camera
from stereo import Stereo
from visualize import SeabornColorPalette

In [39]:
# 指数表記にしない
np.set_printoptions(suppress=True)

Calibrate cameras from calibration points

In [40]:
points_1 = pd.read_csv("points/points_1.csv")
points_2 = pd.read_csv("points/points_2.csv")
points_3 = pd.read_csv("points/points_3.csv")

In [41]:
c1 = Camera(points_1)
c2 = Camera(points_2)
c3 = Camera(points_3)

c1.calibrate()
c2.calibrate()
c3.calibrate()

Calibrate camera...
Perspective Projection Matrix
[[ -40.54122968   40.37259325  -22.09534006 1571.80271692]
 [  14.67419715    5.29694456  -55.32118497 1118.37794573]
 [  -0.0117374    -0.0041147    -0.01115845    1.        ]]
Calibrate camera...
Perspective Projection Matrix
[[ -47.5848755    31.63423375  -18.0047358  1582.83552373]
 [   6.04330571    2.71247349  -55.79891734 1083.35787306]
 [  -0.01209186   -0.00646705   -0.00886434    1.        ]]
Calibrate camera...
Perspective Projection Matrix
[[ -46.84682189   27.79618015  -23.94934871 1673.69448202]
 [  16.76095364   12.07519625  -51.80189787  889.83455189]
 [  -0.00888891   -0.00640756   -0.01209724    1.        ]]


## Optional
### 7. Plot the obtained shape in 3D

In [42]:
import plotly.offline
import plotly.graph_objects as go

In [43]:
plotly.offline.init_notebook_mode(connected=True)

##### New Stereo for plotting

In [44]:
points_ = pd.read_csv("points/points_1_2.csv")
add = pd.read_csv("points/points_1_2_add.csv")
points_12 = pd.concat([points_, add], ignore_index=True)
s_ = Stereo(c1, c2, points_12)
s_.obtain_objects_points_by_stereo()

(X, Y, Z) =  [25.46853018 16.45891675  0.27088423]
(X, Y, Z) =  [24.74322514 17.17460602  0.21459361]
(X, Y, Z) =  [21.9623529  20.02167334  0.32099857]
(X, Y, Z) =  [19.91808882 22.15987022  0.38459758]
(X, Y, Z) =  [16.4237593  25.69960559  0.35387484]
(X, Y, Z) =  [5.25763004 7.23655657 9.16204878]
(X, Y, Z) =  [10.59512338  5.50404891  9.27484466]
(X, Y, Z) =  [9.37281706 8.25184944 9.13307199]
(X, Y, Z) =  [ 8.15256931 11.08960475  9.13429177]
(X, Y, Z) =  [4.45047282 9.9362386  9.156412  ]
(X, Y, Z) =  [11.59571684  6.51458292  2.8439244 ]
(X, Y, Z) =  [10.64963668 10.55205759  6.59275462]
(X, Y, Z) =  [11.65461478  6.63672553  6.33230488]
(X, Y, Z) =  [ 9.52752728 11.54604342  8.79831639]
(X, Y, Z) =  [ 9.39823315 11.52663161  2.80247845]
(X, Y, Z) =  [13.09258375 17.84186136  6.14589095]
(X, Y, Z) =  [15.30964117 17.3783268   1.98313607]
(X, Y, Z) =  [13.30651068 21.42861199  2.55998543]
(X, Y, Z) =  [18.57716264 12.74336519  6.16123568]
(X, Y, Z) =  [ 8.38610995 27.03306098  9

In [45]:
# 物体の情報を付加させる
points_of_objects = s_.points_of_objects
object_list = ["carpenter", "cylinder", "block", "duck", "metal"]
object_ids = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 2, 4,
              4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
points_of_objects["Object"] = [object_list[o] for o in object_ids]
points_of_objects.tail()

,X,Y,Z,Object
38,19.286625,14.725523,5.811751,block
39,20.632519,13.197162,6.234151,block
40,20.603731,13.589286,6.178362,block
41,20.547658,13.952406,6.148863,block
42,20.553605,14.369325,6.169674,block


In [46]:
color_list = ["rgb(244,69,96)", "rgb(1,12,30)", "rgb(247,220,102)", "rgb(255,251,56)", "rgb(25,48,61)"]

data = []

# オブジェクトごとに色を変える
for object_, color_ in zip(object_list, color_list):
    points_ = points_of_objects[points_of_objects["Object"] == object_]


    trace = go.Scatter3d(
        x=points_["X"],
        y=points_["Y"],
        z=points_["Z"],
        mode="markers",
        marker=dict(
            color=color_,
            size=5,
            opacity=1.0
        ),
        name=object_
        )
    data += [trace]


layout=go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    scene=dict(aspectmode="cube"),
    legend=dict(x=0.9, y=0.9)
)

##### Reconstruction of the cylinder from the circle determined by the least squares method

In [47]:
# 円のパラメータ　(report.ipynbで求めたもの)
a, b, r = 7.268304187128073, 7.626023293974528, 4.481011182745071

theta = np.linspace(0, 2 * np.pi, 60)
x_, y_ = a + r * np.cos(theta), b + r * np.sin(theta)

# 円柱の高さを推定値のz座標の平均とする
points_top_ = s_.points_of_objects.iloc[5:10, :]
height_cylinder = np.average(points_top_["Z"])

# z軸方向に何個の円を描画するか
z_step = 20
z_range = np.linspace(0, height_cylinder, z_step)

In [48]:
color_list = SeabornColorPalette.to_plotly_rgb("bone", z_step)

# プロットするデータを作成
data_cylinder = []

for z_, color_ in zip(z_range, color_list):
    trace = go.Scatter3d(
        x=x_,
        y=y_,
        z=[z_] * len(theta),
        marker=dict(
            color=color_,
            size=5,
            opacity=0.3,
            symbol="diamond"
        ),
        name="estimated_cylinder"
    )
    data_cylinder += [trace]

##### Reconstruction of the cuboid from the top four corners of the surface

In [49]:
points_corner_ = s_.points_of_objects.iloc[20:24, :]
points_corner_

# 金属の高さを推定値のz座標の平均とする
height_metal = np.mean(points_corner_["Z"])

z_range = np.linspace(0, height_metal, z_step)

In [50]:
data_metal = []

for z_, color_ in zip(z_range, color_list):
    trace = go.Scatter3d(
        x=list(points_corner_["X"]) + [points_corner_.iat[0, 0]],
        y=list(points_corner_["Y"]) + [points_corner_.iat[0, 1]],
        z=[z_] * (len(points_corner_) + 1),
        marker=dict(
            color=color_,
            size=5,
            opacity=0.3,
            symbol="diamond"
        ),
        name="estimated_metal"
    )
    data_metal += [trace]

In [51]:
fig = go.Figure(data=data + data_cylinder + data_metal, layout=layout)
plotly.offline.plot(fig, filename="plot_3d.html")
fig.show()